In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import gc

In [3]:
os.chdir('/home/luke/Desktop/kaggle/Home_Credit_Default_Risk')
x_val = pd.read_csv('x_val.csv')
y_val = pd.read_csv('y_val.csv')
partial_y_train = pd.read_csv('partial_y_train.csv')
partial_x_train = pd.read_csv('partial_x_train.csv')

In [4]:
print('Training data shape: ', partial_x_train.shape)
print('Training label shape: ', partial_y_train.shape)
print('Validation data shape: ', x_val.shape)
print('Validation label shape: ', y_val.shape)

Training data shape:  (239999, 239)
Training label shape:  (239999, 1)
Validation data shape:  (67510, 239)
Validation label shape:  (67510, 1)


In [4]:
partial_x_train.head()

,2.197455179324805385e-01,0.000000000000000000e+00,1.000000000000000000e+00,0.000000000000000000e+00.1,5.263157894736841813e-02,2.454384230388970047e-03,5.693483146067415679e-02,5.533520533520533413e-02,4.377104377104376609e-02,1.560552770777368613e-02,...,0.000000000000000000e+00.154,0.000000000000000000e+00.155,0.000000000000000000e+00.156,0.000000000000000000e+00.157,0.000000000000000000e+00.158,0.000000000000000000e+00.159,0.000000000000000000e+00.160,0.000000000000000000e+00.161,0.000000000000000000e+00.162,0.000000000000000000e+00.163
0,0.219751,0.0,0.0,0.0,0.000000,0.002089,0.126730,0.094068,0.108866,0.157398,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.219757,0.0,0.0,1.0,0.105263,0.001166,0.056180,0.061495,0.057239,0.093190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.219760,0.0,0.0,1.0,0.052632,0.000935,0.236725,0.158301,0.217733,0.134897,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.219762,0.0,1.0,1.0,0.000000,0.002281,0.388515,0.158915,0.346801,0.129705,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.219765,0.0,1.0,1.0,0.000000,0.001320,0.119333,0.098333,0.098765,0.309272,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [22]:
x_test = pd.read_csv('x_test.csv')

In [23]:
x_test_use = np.array(x_test)

In [24]:
x_test.shape

(48743, 239)

In [ ]:
# Format the training and testing data 
train = np.array(application_train.drop(columns = 'TARGET'))
test = np.array(application_test)

train_labels = np.array(train_labels).reshape((-1, ))

# 10 fold cross validation
folds = KFold(n_splits=5, shuffle=True, random_state=50)

# Validation and test predictions
valid_preds = np.zeros(train.shape[0])
test_preds = np.zeros(test.shape[0])

# Iterate through each fold
for n_fold, (train_indices, valid_indices) in enumerate(folds.split(train)):
    # Training data for the fold
    train_fold, train_fold_labels = train[train_indices, :], train_labels[train_indices]
    
    # Validation data for the fold
    valid_fold, valid_fold_labels = train[valid_indices, :], train_labels[valid_indices]
    
    # LightGBM classifier with hyperparameters
    clf = LGBMClassifier(
        n_estimators=10000,
        learning_rate=0.1,
        subsample=.8,
        max_depth=7,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=2
    )
    
    # Fit on the training data, evaluate on the validation data
    clf.fit(train_fold, train_fold_labels, 
            eval_set= [(train_fold, train_fold_labels), (valid_fold, valid_fold_labels)], 
            eval_metric='auc', early_stopping_rounds=100, verbose = False
           )
    
    # Validation preditions
    valid_preds[valid_indices] = clf.predict_proba(valid_fold, num_iteration=clf.best_iteration_)[:, 1]
    
    # Testing predictions
    test_preds += clf.predict_proba(test, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
    
    # Display the performance for the current fold
    print('Fold %d AUC : %0.6f' % (n_fold + 1, roc_auc_score(valid_fold_labels, valid_preds[valid_indices])))
    
    # Delete variables to free up memory
    del clf, train_fold, train_fold_labels, valid_fold, valid_fold_labels
    gc.collect()
    

# Make a submission dataframe
#submission = application_test[['SK_ID_CURR']]
#submission['TARGET'] = test_preds


In [44]:
from keras import models
from keras import layers
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier

In [13]:
partial_x_train.shape

(239999, 239)

In [38]:
def test(unit,epochs,batch_size):
    model = models.Sequential()
    model.add(layers.Dense(64, input_shape=(239,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(partial_x_train,partial_y_train, epochs=20, verbose=0,batch_size=100,validation_data=(x_val, y_val))
    y_pred = model.predict_proba(x_test_use)
    return roc_auc_score(y_val, model.predict_proba(x_val))

In [39]:
from sklearn.model_selection import GridSearchCV

In [60]:
%%time
# Function to create model, required for KerasClassifier
def create_model(optimizer, init):
    model = models.Sequential()
    model.add(layers.Dense(64, kernel_initializer=init, input_shape=(239,)))
    model.add(layers.Dense(64, kernel_initializer=init, activation='relu'))
    model.add(layers.Dense(1, kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    return(model)
model = KerasClassifier(build_fn=create_model, verbose=0, epochs = 2, batch_size = 5000)

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal']
# epochs = [5,10]
# batches = [1000,5000]
param_grid = dict(optimizer = optimizers, init = init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc')
grid_result = grid.fit(partial_x_train, partial_y_train)

CPU times: user 47.4 s, sys: 2.75 s, total: 50.1 s
Wall time: 40.1 s


In [59]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.704093 using {'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.704093 (0.007345) with: {'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.658728 (0.006197) with: {'init': 'glorot_uniform', 'optimizer': 'adam'}
0.660480 (0.003081) with: {'init': 'uniform', 'optimizer': 'rmsprop'}
0.609402 (0.003451) with: {'init': 'uniform', 'optimizer': 'adam'}


In [30]:
model = models.Sequential()
model.add(layers.Dense(64, input_shape=(239,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
model.fit(partial_x_train,partial_y_train, epochs=20, batch_size=100,validation_data=(x_val, y_val))

Train on 239999 samples, validate on 67510 samples
Epoch 1/20
239999/239999 [==============================] - 5s 21us/step - loss: 0.2579 - acc: 0.9189 - val_loss: 0.2517 - val_acc: 0.9195
Epoch 2/20
239999/239999 [==============================] - 5s 20us/step - loss: 0.2528 - acc: 0.9193 - val_loss: 0.2505 - val_acc: 0.9196
Epoch 3/20
239999/239999 [==============================] - 5s 20us/step - loss: 0.2517 - acc: 0.9193 - val_loss: 0.2517 - val_acc: 0.9196
Epoch 4/20
239999/239999 [==============================] - 5s 20us/step - loss: 0.2511 - acc: 0.9193 - val_loss: 0.2509 - val_acc: 0.9194
Epoch 5/20
239999/239999 [==============================] - 5s 20us/step - loss: 0.2509 - acc: 0.9193 - val_loss: 0.2506 - val_acc: 0.9190
Epoch 6/20
239999/239999 [==============================] - 5s 20us/step - loss: 0.2503 - acc: 0.9194 - val_loss: 0.2506 - val_acc: 0.9193
Epoch 7/20
239999/239999 [==============================] - 5s 20us/step - loss: 0.2501 - acc: 0.9195 - val_loss: 0

In [32]:
y_pred = model.predict_proba(x_test_use)
y_pred

array([[0.31712615],
       [0.05558179],
       [0.04675853],
       ...,
       [0.05032958],
       [0.0774654 ],
       [0.11556797]], dtype=float32)

In [33]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, model.predict_proba(x_val))

0.7327954279091746

In [42]:
model = models.Sequential()
model.add(layers.Dense(64, input_shape=(239,)),kernel_regularizer=regularizers.l2(0.01))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])

model.fit(partial_x_train,partial_y_train,epochs = 10,batch_size=100000,validation_data = (x_val,y_val),verbose = 1)
y_pred = model.predict_proba(x_test)

TypeError: add() got an unexpected keyword argument 'kernel_regularizer'